# 🧠 Waste Detection Project – Model Training & Inference

Welcome to the **Model Training & Inference** stage of the Waste Detection AI Workshop. In this notebook, we’ll prepare the data and train a YOLOv11 model to detect different types of waste from images.

In this notebook, we:
- Split the dataset into train, validation, and test sets.
- Create a data.yaml config file for YOLO.
- Train a YOLO model using the Ultralytics package.
- Run inference on test images.

📁 **Before you begin**, make sure your dataset (after augmentation) is organized as follows:
```
Garbage-Dataset/
├── augmented_images/
│ ├── biological_001.jpeg
│ ├── biological_002.jpeg
│ ├── ...
├── augmented_labels/
│ ├── biological_001.txt
│ ├── biological_002.txt
│ ├── ...
```

> ✅ Make sure the EDA and Data Augmentation steps are complete before proceeding.

## 📥 Install Required Packages

In [ ]:
# Install required packages for splitting and YOLO
!pip install scikit-learn ultralytics

## Split Dataset into Train / Val / Test

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Original dataset (augmented)
image_dir = 'Garbage-Dataset/augmented_images/'
label_dir = 'Garbage-Dataset/augmented_labels/'

# Target split folders
train_image_dir = 'Garbage-Dataset/train/images/'
val_image_dir = 'Garbage-Dataset/val/images/'
test_image_dir = 'Garbage-Dataset/test/images/'

train_label_dir = 'Garbage-Dataset/train/labels/'
val_label_dir = 'Garbage-Dataset/val/labels/'
test_label_dir = 'Garbage-Dataset/test/labels/'

# Create necessary directories
for path in [train_image_dir, val_image_dir, test_image_dir, train_label_dir, val_label_dir, test_label_dir]:
    os.makedirs(path, exist_ok=True)

# Get image and label files
image_files = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
label_files = [f.rsplit('.', 1)[0] + '.txt' for f in image_files]

# Ensure all label files exist
assert all(os.path.exists(os.path.join(label_dir, f)) for f in label_files), "Missing label files"

# Split: 70% train, 20% val, 10% test
train_imgs, temp_imgs = train_test_split(image_files, test_size=0.3, random_state=42)
val_imgs, test_imgs = train_test_split(temp_imgs, test_size=1/3, random_state=42)

# Move files
def move_files(images, src_img_dir, src_lbl_dir, dst_img_dir, dst_lbl_dir):
    for img in images:
        base = img.rsplit('.', 1)[0]
        lbl = base + '.txt'
        shutil.copy(os.path.join(src_img_dir, img), os.path.join(dst_img_dir, img))
        shutil.copy(os.path.join(src_lbl_dir, lbl), os.path.join(dst_lbl_dir, lbl))

move_files(train_imgs, image_dir, label_dir, train_image_dir, train_label_dir)
move_files(val_imgs, image_dir, label_dir, val_image_dir, val_label_dir)
move_files(test_imgs, image_dir, label_dir, test_image_dir, test_label_dir)

print("✅ Dataset split complete: Train (70%), Val (20%), Test (10%)")

## Create data.yaml for YOLO

In [ ]:
data_yaml = """
path: Garbage-Dataset
train: train/images
val: val/images
test: test/images

nc: 7
names:
  0: 'biological'
  1: 'cardboard'
  2: 'glass'
  3: 'metal'
  4: 'paper'
  5: 'plastic'
  6: 'trash'
"""

with open("data.yaml", "w") as f:
    f.write(data_yaml)

## 🚨 Train YOLO Model
You can change the model to any of the following depending on your system's power:
```
- yolo11n.pt (nano - fastest)
- yolo11s.pt (small)
- yolo11m.pt (medium)
- yolo11l.pt (large)
- yolo11x.pt (extra large)
```

```pip install numpy==1.26.4``` (ignore)

In [ ]:
!yolo detect train data=data.yaml model=yolo11n.pt epochs=2 imgsz=640

## 🔍 Run Inference on a Test Image

In [ ]:
from IPython.display import Image

# Display the original image before running inference
Image(filename='Garbage-Dataset/test/images/aug_0_cardboard_032.jpg')

In [ ]:
!yolo detect predict model=runs/detect/train/weights/best.pt source=Garbage-Dataset/test/images/aug_0_cardboard_032.jpg
Image(filename='runs/detect/predict/aug_0_cardboard_032.jpg')

## ✅ Model Training & Inference Completed!

### You’ve now successfully trained a YOLO model and run inference on a sample image.

#### 📦 **Next Step**: Proceed to the garbage-detection-app directory to integrate the trained model into a GUI for deployment.

#### Make sure the trained model weights (best.pt) are copied into the app folder